### Tensor
- PyTorch의 핵심 데이터 구조
- Scalar, Vector(1D), Matrix(2D), 고차원 배열(ND) 까지 모두 표현 가능
- NumPy의 ndarray와 비슷하지만, GPU에서 연산 가능하고 자동 미분(autograd)도 지원


In [1]:
import torch
import numpy as np

#### Tensor 초기화

1. 데이터로부터 직접(directly) 생성
- 데이터의 자료형(data type)은 자동으로 유추

In [2]:
data = [[1, 2], [3, 4]]
x_data = torch.tensor(data)

2. NumPy 배열로부터 생성
- Tensor to NumPy, NumPy to Tensor 가능

In [3]:
np_array = np.array(data)
x_np = torch.from_numpy(np_array)

3. 다른 텐서로부터 생성하기
- 명시적으로 재정의(override)하지 않는다면, 인자로 주어진 텐서의 속성 (모양(shape), 자료형(datatype))을 유지

In [4]:
x_ones = torch.ones_like(x_data)        # x_data와 같은 shape을 갖는 텐서를 만들되, 모든 원소의 값을 1로 채움
print(f"Ones Tensor: \n {x_ones} \n")

x_rand = torch.rand_like(x_data, dtype=torch.float)     # x_data와 같은 shape을 갖는 텐서를 만들되, 0이상 1미만의 난수 텐서를 생성 / float형
print(f"Random Tensor: \n {x_rand} \n") 

x_randint = torch.randint(low=0, high=10, size=x_data.shape, dtype=torch.int32) # 정수형 난수
print(f"Random Tensor(int): \n {x_randint} \n")

Ones Tensor: 
 tensor([[1, 1],
        [1, 1]]) 

Random Tensor: 
 tensor([[0.0158, 0.1336],
        [0.1653, 0.0300]]) 

Random Tensor(int): 
 tensor([[3, 4],
        [2, 0]], dtype=torch.int32) 



4. 무작위(random) 또는 상수(constant) 값을 사용하기
- shape은 텐서의 차원(dimension)을 나타내는 튜플(tuple)로, 아래 함수들에서는 출력 텐서의 차원을 결정함

In [5]:
shape = (2, 3,)
rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)

print(f"Random Tensor: \n {rand_tensor} \n")
print(f"Ones Tensor: \n {ones_tensor} \n")
print(f"Zeros Tensor: \n {zeros_tensor}")

Random Tensor: 
 tensor([[0.4971, 0.1129, 0.7153],
        [0.8702, 0.1793, 0.2449]]) 

Ones Tensor: 
 tensor([[1., 1., 1.],
        [1., 1., 1.]]) 

Zeros Tensor: 
 tensor([[0., 0., 0.],
        [0., 0., 0.]])


#### 텐서의 속성 (Attribute)
- 텐서의 속성은 텐서의 모양(shape), 자료형(datatype) 및 어느 장치에 저장되는지를 나타냄

In [6]:
tensor = torch.rand(3, 4)

print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on : {tensor.device}")

Shape of tensor: torch.Size([3, 4])
Datatype of tensor: torch.float32
Device tensor is stored on : cpu


#### 텐서 연산 (Operation)

In [7]:
if torch.cuda.is_available():
    tensor = tensor.to('cuda')

NumPy식의 표준 인덱싱과 슬라이싱

In [8]:
tensor = torch.arange(20).reshape(4, 5)
print(f"Tensor: {tensor}")
print(f"First row: {tensor[0]}")
print(f"First columns: {tensor[:, 0]}")
# ... : 마지막 차원
print(f"Last columns: {tensor[..., -1]}")   # 2차원에서는 :와 동일하게 작용
tensor[:, 1] = 0
print(tensor)

Tensor: tensor([[ 0,  1,  2,  3,  4],
        [ 5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19]])
First row: tensor([0, 1, 2, 3, 4])
First columns: tensor([ 0,  5, 10, 15])
Last columns: tensor([ 4,  9, 14, 19])
tensor([[ 0,  0,  2,  3,  4],
        [ 5,  0,  7,  8,  9],
        [10,  0, 12, 13, 14],
        [15,  0, 17, 18, 19]])


텐서 합치기

In [9]:
t1 = torch.cat([tensor, tensor, tensor], dim=1) # dim=1: row를 기준으로 (batch, row, column)
print(t1)

tensor([[ 0,  0,  2,  3,  4,  0,  0,  2,  3,  4,  0,  0,  2,  3,  4],
        [ 5,  0,  7,  8,  9,  5,  0,  7,  8,  9,  5,  0,  7,  8,  9],
        [10,  0, 12, 13, 14, 10,  0, 12, 13, 14, 10,  0, 12, 13, 14],
        [15,  0, 17, 18, 19, 15,  0, 17, 18, 19, 15,  0, 17, 18, 19]])


산술 연산 (Arithmetic operations)

In [10]:
tensor = torch.arange(20).reshape(4, 5).float()
print(tensor)

# 두 텐서 간의 행렬 곱(matrix multiplication)을 계산. y1, y2, y3은 모두 같은 값을 가짐
y1 = tensor @ tensor.T
y2 = tensor.matmul(tensor.T)

y3 = torch.rand_like(y1)                # 행렬곱 결과를 저장할 그릇 (메모리 확보)
torch.matmul(tensor, tensor.T, out=y3)  # out=y3 -> 새로 메모리를 만들지 않고 y3에 덮어써서 저장
print(y1)

# element-wise product를 계산. z1, z2, z3는 모두 같은 값을 가짐
z1 = tensor * tensor
z2 = tensor.mul(tensor)

z3 = torch.rand_like(tensor)
torch.mul(tensor, tensor, out=z3)
print(z1)

tensor([[ 0.,  1.,  2.,  3.,  4.],
        [ 5.,  6.,  7.,  8.,  9.],
        [10., 11., 12., 13., 14.],
        [15., 16., 17., 18., 19.]])
tensor([[  30.,   80.,  130.,  180.],
        [  80.,  255.,  430.,  605.],
        [ 130.,  430.,  730., 1030.],
        [ 180.,  605., 1030., 1455.]])
tensor([[  0.,   1.,   4.,   9.,  16.],
        [ 25.,  36.,  49.,  64.,  81.],
        [100., 121., 144., 169., 196.],
        [225., 256., 289., 324., 361.]])


단일-요소(single-element) 텐서


In [11]:
agg = tensor.sum()
agg_item = agg.item()
print(agg_item, type(agg.item))

190.0 <class 'builtin_function_or_method'>


바꿔치기(in-place) 연산

In [12]:
print(f"{tensor} \n")
tensor.add(5)
print(tensor)

tensor([[ 0.,  1.,  2.,  3.,  4.],
        [ 5.,  6.,  7.,  8.,  9.],
        [10., 11., 12., 13., 14.],
        [15., 16., 17., 18., 19.]]) 

tensor([[ 0.,  1.,  2.,  3.,  4.],
        [ 5.,  6.,  7.,  8.,  9.],
        [10., 11., 12., 13., 14.],
        [15., 16., 17., 18., 19.]])


#### NumPy 변환 (Bridge)
- CPU 상의 tensor와 NumPy 배열은 메모리 공간을 공유하기 때문에, 하나를 변경하면 다른 하나도 변경됨 (shallow copy와 유사)

텐서를 NumPy 배열로 변환하기

In [13]:
t = torch.ones(5)
print(f"t: {t}")
n = t.numpy()
print(f"n: {n}")

t: tensor([1., 1., 1., 1., 1.])
n: [1. 1. 1. 1. 1.]


In [14]:
"""
add와 add_

add(): 더한 결과를 새로 반환 (원본 텐서 변경 X)
add_() : 원본에 직접 더함(in-place) (원본 텐서 변경)
"""

t.add_(1)
print(f"t: {t}")
print(f"n: {n}")        # 메모리 공간을 공유 -> t를 변경하면 n도 변경

t: tensor([2., 2., 2., 2., 2.])
n: [2. 2. 2. 2. 2.]


NumPy 배열을 텐서로 변환하기


In [15]:
n = np.ones(5)
t = torch.from_numpy(n)

마찬가지로, NumPy 배열의 변경 사항이 텐서에 반영

In [16]:
np.add(n, 1, out=n)
print(f"t: {t}")
print(f"n: {n}")

t: tensor([2., 2., 2., 2., 2.], dtype=torch.float64)
n: [2. 2. 2. 2. 2.]
